In [2]:
from utils import utils
from glob import glob
from imp import reload
from tqdm import tqdm
import codecs
import os
reload(utils)

<module 'utils.utils' from '/Users/wanjun/Desktop/比赛训练营/DataFoutain隐私识别/DBC_code/src/utils/utils.py'>

In [135]:
tdata_lst = glob('../data/train_data/*.txt')
symbol = ['。','!','！','?','？','﹗']

In [136]:
# 生成样本的参数
symbol = ['？','⋯','…','﹗']
max_sent_length = 250
max_input_length = 300       

## 划分训练集和验证集

In [137]:
from sklearn.model_selection import train_test_split
from glob import glob
data_lst = glob('../data/train_data/*.txt')
train_lst,val_lst = train_test_split(data_lst,test_size=0.1,random_state=0)
print('train:{}'.format(len(train_lst)))
print('val:{}'.format(len(val_lst)))

train:2263
val:252


In [138]:
data_dir = '../data/train_label'
for data_path in tqdm(train_lst):
    file_name = os.path.basename(data_path).split('.')[0]
    label_path = file_name+'.csv'
    label_path = os.path.join(data_dir,label_path)
    output_path = '../data/train_new'
    output_file = file_name
    utils.gen_data(label_path, data_path, output_path, output_file,symbol,max_sent_length,max_input_length,bioes = False)
for data_path in tqdm(val_lst):
    file_name = os.path.basename(data_path).split('.')[0]
    label_path = file_name+'.csv'
    label_path = os.path.join(data_dir,label_path)
    output_path = '../data/val_new'
    output_file = file_name
    utils.gen_data(label_path, data_path, output_path, output_file,symbol,max_sent_length,max_input_length,bioes = False)

100%|██████████| 252/252 [00:01<00:00, 243.54it/s]


## 文本数据增强
[`nlpcda`](https://github.com/425776024/nlpcda)

In [139]:
# 需要进行增强的标签
labels = [
    'position',
    'name',
    'organization',
    'company',
    'address',
    'movie',
    'game',
    'government',
    'scene',
    'book',
    'mobile',
    'email',
    'QQ',
    'vx',
]

In [140]:
# 对数据进行检查，数据要满足格式 xxx\tO\n 的形式
for filename in glob('../data/train_new/*.txt'):
    utils.check_aug(filename)

../data/train_new/1697-3-new.txt
../data/train_new/1697-2-new.txt
../data/train_new/1697-0-new.txt
../data/train_new/1697-1-new.txt


In [146]:
from nlpcda import Ner
# ner_dir_name: 在ner数据放在ner_data目录下（里面很多.txt），BIO 标记格式
# ignore_tag_list=['O'] : 数据里面O标签的不需要管
# data_augument_tag_list=['P', 'LOC'] : 需要增强的实体
# augument_size=3 : 每条标注数据，最多新增强数量
# seed=0 : 随机种子/ 可缺省
# 原理：把目录中的实体抽出来，然后对同类实体进行随机的替换。
augument_size = 3
ner = Ner(ner_dir_name='../data/train_new/',
        ignore_tag_list=['O'],
        data_augument_tag_list=labels,
        augument_size=augument_size, seed=0)

In [162]:
for filename in tqdm(glob('../data/train_new/*.txt')):
    data_sentence_arrs, data_label_arrs = ner.augment(file_name=filename)
    if len(data_sentence_arrs) > 0:
        for aug_index in range(len(data_sentence_arrs)+1):
            output_path_ = os.path.join('../data/train_aug/','aug-{}-{}'.format(aug_index,os.path.basename(filename)))
            # 拷贝原始数据
            if aug_index == len(data_sentence_arrs):
                with codecs.open(output_path_, "w", encoding="utf-8") as w:
                    raw_file = utils.read_txt(filename)
                    for line in raw_file:
                        w.write(line)
                    w.write('{}\n'.format('END\tO'))
            # 数据增强写入
            else:
                with codecs.open(output_path_, "w", encoding="utf-8") as w:
                    for i,j in zip(data_sentence_arrs[aug_index],data_label_arrs[aug_index]):
                        line = '{}\t{}\n'.format(i,j)
                        w.write(line)
                    w.write('{}\n'.format('END\tO'))

100%|██████████| 2478/2478 [00:06<00:00, 411.82it/s]


## 数据合并成 txt

In [176]:
# 训练集合并
# 写入数据增强
w_path = "../data/train_aug.txt"
for file in os.listdir('../data/train_aug/'):
    path = os.path.join("../data/train_aug", file)
    if not file.endswith(".txt"):  
        continue
    q_list = []
    print("开始读取文件:%s" % file)
    with open(path, "r", encoding="utf-8") as f:
        line = f.readline()
        line = line.strip("\n\r")
        while line != "END\tO":
            q_list.append(line)
            line = f.readline()
            line = line.strip("\n\r")
    print("开始写入文本%s" % w_path)
    with codecs.open(w_path, "a", encoding="utf-8") as f:
        for item in q_list:
            if item.__contains__('\ufeff1'):
                print("===============")
            f.write('%s\n' % item)
        f.write('\n')

开始读取文件:aug-2-1898-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1953-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-866-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-2441-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-471-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1448-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-2069-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-2312-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-192-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1733-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1219-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1481-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1162-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-782-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-261-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-84-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-2296-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1354-0-new.txt
开始写入文本../data/train_aug.txt

开始写入文本../data/train_aug.txt
开始读取文件:aug-0-2424-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-245-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-2377-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1756-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-842-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-455-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-559-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1560-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-2141-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1083-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-2188-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1331-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-590-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-987-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-1886-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-2496-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-421-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1305-0-new.txt

开始写入文本../data/train_aug.txt
开始读取文件:aug-1-2430-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1011-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1689-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1640-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-2261-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-502-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-124-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1820-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-819-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-2057-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1476-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1227-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-39-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-636-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-2155-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-638-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-312-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1509-2-new.txt


开始写入文本../data/train_aug.txt
开始读取文件:aug-3-781-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-1317-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-262-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1546-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-748-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-2484-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-2065-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1894-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-600-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1955-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-324-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-2314-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1735-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1164-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1487-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-226-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1818-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-821-0-new.txt

开始读取文件:aug-2-1309-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1493-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-232-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1170-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-2300-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-718-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1721-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1843-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-614-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1988-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-716-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-10-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1941-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1849-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-13-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-752-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1880-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-650-0-new.txt
开始写入文本../data/train_aug.txt
开始

...

开始读取文件:aug-2-82-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1913-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-2487-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-430-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-1545-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-2164-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1314-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1216-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-827-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-1897-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-829-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-578-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1956-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1634-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-576-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-863-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-474-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-2317-0-new.txt
开始写入文本../data/train_aug.txt
开

开始写入文本../data/train_aug.txt
开始读取文件:aug-2-812-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-1570-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-2151-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-1093-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-405-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-2198-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1321-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-679-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-353-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1860-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-1228-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1153-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-1600-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1098-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-296-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-2221-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-775-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-2470-0-new.tx

开始读取文件:aug-1-946-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-883-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1873-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-494-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-177-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-767-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-2004-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1425-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-284-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1274-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-388-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-341-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1935-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-800-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-2427-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-417-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-2374-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1755-0-new.txt
开始写入文本../data/train_aug.txt
开始

开始写入文本../data/train_aug.txt
开始读取文件:aug-3-2139-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-1380-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1518-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-716-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-1987-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-106-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-2074-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-1455-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-520-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1204-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-937-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-108-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-2304-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-1725-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-650-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1174-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-1497-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-699-0-new.txt

开始读取文件:aug-2-1357-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1950-1-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-156-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1852-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-967-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-2168-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1549-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-570-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1632-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-2213-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1318-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1580-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1063-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-1-2218-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-2480-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-0-600-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-2-1639-0-new.txt
开始写入文本../data/train_aug.txt
开始读取文件:aug-3-1542-0-new.txt
开始写入文本../data/train_aug.

In [5]:
# 验证集增强（这里不做任何操作）
for file in os.listdir('../data/val_new/'):
    filename = os.path.join("../data/val_new", file)
    if not filename.endswith(".txt"):  
        continue
    output_path_ = os.path.join('../data/val_aug/','aug-{}-{}'.format(0,os.path.basename(filename)))
    with codecs.open(output_path_, "w", encoding="utf-8") as w:
        raw_file = utils.read_txt(filename)
        for line in raw_file:
            w.write(line)
        w.write('{}\n'.format('END\tO'))
# 验证集合并
w_path = "../data/val_aug.txt"
for file in os.listdir('../data/val_aug/'):
    path = os.path.join("../data/val_aug", file)
    if not file.endswith(".txt"):  
        continue
    q_list = []
    print("开始读取文件:%s" % file)
    with codecs.open(path, "r", encoding="utf-8") as f:
        line = f.readline()
        line = line.strip("\n\r")
        while line != "END\tO":
            q_list.append(line)
            line = f.readline()
            line = line.strip("\n\r")
    print("开始写入文本%s" % w_path)
    with codecs.open(w_path, "a", encoding="utf-8") as f:
        for item in q_list:
            if item.__contains__('\ufeff1'):
                print("===============")
            f.write('%s\n' % item)
        f.write('\n')
    f.close()

开始读取文件:aug-0-1899-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1978-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-2240-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-322-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-734-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-444-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-2111-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-385-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-2333-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-2513-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1787-1-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-697-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-2166-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-119-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1421-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-243-1-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-90-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-2172-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1052-0-new.txt
开始写入文本../

开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1111-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1915-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1312-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1581-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1548-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-534-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-2257-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-746-1-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-644-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-778-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-2084-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1422-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1458-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1805-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-2471-1-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-2059-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1001-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件:aug-0-1237-0-new.txt
开始写入文本../data/val_aug.txt
开始读取文件